In [0]:
!pip list
!pip install requests feedparser

Package                            Version
---------------------------------- ---------------
annotated-doc                      0.0.4
annotated-types                    0.7.0
anyio                              4.6.2
argon2-cffi                        21.3.0
argon2-cffi-bindings               21.2.0
arrow                              1.3.0
asttokens                          2.0.5
astunparse                         1.6.3
async-lru                          2.0.4
attrs                              24.3.0
azure-common                       1.1.28
azure-core                         1.34.0
azure-identity                     1.20.0
azure-mgmt-core                    1.5.0
azure-mgmt-web                     8.0.0
azure-storage-blob                 12.23.0
azure-storage-file-datalake        12.17.0
babel                              2.16.0
beautifulsoup4                     4.12.3
black                              24.10.0
bleach                             6.2.0
blinker                        

In [0]:
import requests
import feedparser
from urllib.parse import quote
from datetime import datetime, timezone
from zoneinfo import ZoneInfo
import re

### Definición de lista blanca de dominios (GDELT / RSS)

En esta celda se define `WHITELIST_DOMAINS`, un conjunto de dominios considerados **fuentes financieras y periodísticas fiables**

Esta lista se utiliza posteriormente para:

- Filtrar los resultados devueltos por GDELT y otras fuentes RSS.
- Asegurar que solo se tienen en cuenta medios de comunicación relevantes para el análisis financiero del presente trabajo.
- Reducir ruido y posibles sesgos derivados de fuentes de baja calidad.
- Consulta a sitios no confiables.

Esta decisión de diseño forma parte del control de calidad de los datos de entrada al modelo de sentimiento.


In [0]:
#whitelists para GDELT
WHITELIST_DOMAINS = {
    "reuters.com",
    "bloomberg.com",
    "wsj.com",
    "ft.com",
    "cnbc.com",
    "finance.yahoo.com",
    "marketwatch.com",
    "investing.com",
    "seekingalpha.com",
    "fool.com",
    "morningstar.com",
    "thestreet.com",
    "barrons.com",
    "businessinsider.com",
    "apnews.com",
    "abcnews.go.com",
    "nytimes.com",
    "washingtonpost.com",
}

def domain_allowed(domain: str, whitelist: set[str]) -> bool:
    if not domain:
        return False
    d = domain.lower().strip()
    return any(d == w or d.endswith("." + w) for w in whitelist)

# Declaración de los helpers de preprocesado

- Se define la zona horaria de referencia `MADRID_TZ` (Europa/Madrid), coherente con el contexto del presente trabajo.
- Se implementan funciones auxiliares (_helpers_) para limpiar HTML, normalizar textos y manipular campos temporales,
que se reutilizan en las funciones de obtención de noticias.

In [0]:
MADRID_TZ = ZoneInfo("UTC")

# ---------------------------
# Helpers
# ---------------------------
def _clean_html(text: str) -> str:
    if not text:
        return ""
    return re.sub(r"<[^>]+>", "", text).strip()


def _today_local(tz=MADRID_TZ) -> datetime.date:
    return datetime.now(tz).date()


def _parse_gdelt_seendate(seendate: str):
    """
    Soporta ambos formatos que puede devolver GDELT:
    - 'YYYYMMDDHHMMSS' (antiguo)
    - 'YYYYMMDDTHHMMSSZ' o 'YYYYMMDDT HHMMSS Z' (ej: 20260106T073000Z)
    Devuelve datetime en UTC con tzinfo.
    """
    if not seendate:
        return None

    s = seendate.strip()

    # Formato: 20260106T073000Z
    try:
        if "T" in s and s.endswith("Z"):
            dt_utc = datetime.strptime(s, "%Y%m%dT%H%M%SZ").replace(tzinfo=timezone.utc)
            return dt_utc
    except ValueError:
        pass

    # Formato: 20260106073000
    try:
        dt_utc = datetime.strptime(s, "%Y%m%d%H%M%S").replace(tzinfo=timezone.utc)
        return dt_utc
    except ValueError:
        return None



### Cliente RSS de GDELT: extracción de noticias recientes

Esta celda implementa la lógica para consumir el endpoint de documentos de **GDELT** y obtener noticias recientes relacionadas con una consulta (`query`):

- Construye la URL a la API de GDELT (`/api/v2/doc/doc`).
- Ajusta la ventana temporal de interés (`hours`), típica de 24–48 h, para capturar noticias recientes.
- Aplica funciones de limpieza y normalización de fechas a la salida.

Esta funcionalidad proporciona una de las dos patas del enfoque híbrido de obtención de noticias (GDELT + Google News RSS) utilizado en el TFM.

In [0]:
# ---------------------------
# GDELT


from datetime import timedelta

def fetch_gdelt_last_hours(query: str, n: int = 3, hours: int = 48, sourcelang: str = "eng", tz=MADRID_TZ, asset: str | None = None):
    url = "https://api.gdeltproject.org/api/v2/doc/doc"

    # Sanitizar query para GDELT
    query = query.replace("&", " and ")

    now_utc = datetime.now(timezone.utc)
    start_utc = now_utc - timedelta(hours=hours)

    params = {
        "query": query,
        "mode": "ArtList",
        "format": "JSON",
        "maxrecords": 250,
        "sort": "datedesc",
        "startdatetime": start_utc.strftime("%Y%m%d%H%M%S"),
        "enddatetime": now_utc.strftime("%Y%m%d%H%M%S"),
    }
    if sourcelang:
        params["sourcelang"] = sourcelang

    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json,*/*"}

    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()

    ct = (r.headers.get("content-type") or "").lower()
    if "json" not in ct:
        raise RuntimeError(f"GDELT no devolvió JSON (ct={ct}): {r.text[:200]!r}")

    data = r.json()

    results = []
    seen_titles = set()

    for art in data.get("articles", []) or []:
        title = (art.get("title") or "").strip()
        link = (art.get("url") or "").strip()
        domain = (art.get("domain") or "").lower().strip()
        seendate = art.get("seendate")

        if not title or not link:
            continue

        # dedup por título
        tkey = title.lower()
        if tkey in seen_titles:
            continue
        seen_titles.add(tkey)

        # Parse fecha
        dt_utc = _parse_gdelt_seendate(seendate)
        published_utc = dt_utc.isoformat() if dt_utc else None
        published_local = dt_utc.astimezone(tz).isoformat() if dt_utc else None

        results.append({
            "provider": "gdelt",
            "asset": asset,
            "title": title,
            "link": link,
            "source": art.get("source"),
            "domain": domain,
            "seendate": seendate,
            "published_utc": published_utc,
            "published_local": published_local,
            "summary": art.get("summary") or None,
        })

        if len(results) >= n:
            break

    return results



In [0]:

def fetch_news_gdelt_by_date_range(
    user_input: str,
    start_date: str,          # "YYYY-MM-DD" en hora Madrid
    end_date: str,            # "YYYY-MM-DD" en hora Madrid (incluyente si include_end=True)
    n: int = 50,
    gdelt_lang: str = "eng",
    tz=MADRID_TZ,
    include_end: bool = True,
    require_market: bool = True,   # si lo pones False, no añade AND market
):
    """
    Obtiene noticias de GDELT en un rango de fechas (interpretadas en hora local `tz`).

    Parámetros
    ----------
    user_input : str
        Clave (p.ej. TSLA, BTC, IBEX35) o texto libre (Tesla, Bitcoin, etc.)
    start_date, end_date : str
        Fechas en formato "YYYY-MM-DD" (hora local tz). Ej: "2026-02-01"
    n : int
        Máximo número de resultados a devolver (deduplicado por título).
    include_end : bool
        Si True, incluye el día end_date completo.
    require_market : bool
        Si True añade AND market (más financiero, menos ruido).

    Returns
    -------
    dict con "items" compatible con el resto del notebook.
    """

    # 1) Parse fechas locales -> datetimes locales
    y1, m1, d1 = map(int, start_date.split("-"))
    y2, m2, d2 = map(int, end_date.split("-"))

    start_local = datetime(y1, m1, d1, 0, 0, 0, tzinfo=tz)

    if include_end:
        # hasta el final del día end_date
        end_local = datetime(y2, m2, d2, 23, 59, 59, tzinfo=tz)
    else:
        # inicio del día end_date (end exclusivo)
        end_local = datetime(y2, m2, d2, 0, 0, 0, tzinfo=tz)

    # 2) Convertir a UTC para GDELT
    start_utc = start_local.astimezone(timezone.utc)
    end_utc = end_local.astimezone(timezone.utc)

    # 3) Construir query (reutiliza tu build_query)
    asset = user_input.strip().upper().replace(" ", "")
    base = build_query(user_input)

    # OJO: ya corregiste el tema de paréntesis en GDELT, así que NO envolvemos base
    query_gdelt = f"sourcelang:english {base}"
    if require_market:
        query_gdelt += " AND market"

    # 4) Llamada a GDELT (misma lógica que tu fetch_gdelt_last_hours pero con fechas)
    url = "https://api.gdeltproject.org/api/v2/doc/doc"

    params = {
        "query": query_gdelt.replace("&", " and "),
        "mode": "ArtList",
        "format": "JSON",
        "maxrecords": 250,
        "sort": "datedesc",
        "startdatetime": start_utc.strftime("%Y%m%d%H%M%S"),
        "enddatetime": end_utc.strftime("%Y%m%d%H%M%S"),
    }
    if gdelt_lang:
        params["sourcelang"] = gdelt_lang

    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json,*/*"}

    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()

    ct = (r.headers.get("content-type") or "").lower()
    if "json" not in ct:
        raise RuntimeError(f"GDELT no devolvió JSON (ct={ct}): {r.text[:200]!r}")

    data = r.json()

    results = []
    seen_titles = set()

    for art in data.get("articles", []) or []:
        title = (art.get("title") or "").strip()
        link = (art.get("url") or "").strip()
        if not title or not link:
            continue

        tkey = title.lower()
        if tkey in seen_titles:
            continue
        seen_titles.add(tkey)

        dt_utc = _parse_gdelt_seendate(art.get("seendate"))
        results.append({
            "provider": "gdelt",
            "asset": asset,
            "title": title,
            "link": link,
            "source": art.get("source"),
            "domain": (art.get("domain") or "").lower().strip(),
            "seendate": art.get("seendate"),
            "published_utc": dt_utc.isoformat() if dt_utc else None,
            "published_local": dt_utc.astimezone(tz).isoformat() if dt_utc else None,
            "summary": art.get("summary") or None,
        })

        if len(results) >= n:
            break

    return {
        "source": "gdelt",
        "asset": asset,
        "query": query_gdelt,
        "range_local": {"start": start_local.isoformat(), "end": end_local.isoformat()},
        "range_utc": {"start": start_utc.isoformat(), "end": end_utc.isoformat()},
        "count_returned": len(results),
        "items": results,
    }

### Construcción de queries semánticas para índices bursátiles y activos

La función `build_query` transforma una entrada simple del usuario (por ejemplo, `SP500`, `NASDAQ100`, `IBEX35`) en una **query expandida** apta para:

- GDELT (parámetros de búsqueda avanzados).
- Feeds RSS (Google News, agregadores financieros).

Características clave:

- Mapea alias conocidos (SP500, S&P500, NASDAQ100, IBEX35, etc.) a expresiones con sinónimos, tickers y ETFs representativos.
- Permite capturar distintas formas en las que la prensa financiera se refiere al mismo índice.
- Devuelve una cadena lista para ser utilizada en las funciones de consulta posteriores.

Esta capa de aliasing mejora la **recuperación de noticias relevantes** para el activo/índice de interés en el marco del TFM.

In [0]:
def load_aliases_from_txt(path: str) -> dict[str, str]:
    aliases = {}
    with open(path, encoding="utf-8") as f:
        for line in f:
            if "=" not in line:
                continue
            key, value = line.split("=", 1)
            aliases[key.strip().upper()] = value.strip()
    return aliases


In [0]:
# Diccionario de aliases definido directamente en el código
ALIASES = {
    "TSLA": '("Tesla" OR "Tesla Inc" OR "Tesla stock" OR "Tesla shares" OR "Tesla electric vehicles")',
    "NVDA": '("Nvidia" OR "NVIDIA Corporation" OR "Nvidia stock" OR "Nvidia shares" OR "Nvidia semiconductors")',
    "AMD": '("Advanced Micro Devices" OR "AMD stock" OR "AMD shares" OR "AMD semiconductors")',
    "COIN": '("Coinbase" OR "Coinbase Global" OR "Coinbase stock" OR "Coinbase cryptocurrency exchange")',
    "PLTR": '("Palantir" OR "Palantir Technologies" OR "Palantir stock" OR "Palantir data analytics")',
    "RIVN": '("Rivian" OR "Rivian Automotive" OR "Rivian stock" OR "Rivian electric vehicles")',
    "SHOP": '("Shopify" OR "Shopify Inc" OR "Shopify stock" OR "Shopify e-commerce platform")',
    "LCID": '("Lucid Motors" OR "Lucid Group" OR "Lucid stock" OR "Lucid electric vehicles")',
    "ZM": '("Zoom Video Communications" OR "Zoom stock" OR "Zoom video conferencing")',
    "SPCE": '("Virgin Galactic" OR "Virgin Galactic Holdings" OR "Virgin Galactic stock" OR "space tourism company")',
    "KO": '("Coca-Cola" OR "Coca-Cola Company" OR "Coca-Cola stock" OR "Coca-Cola beverages")',
    "PG": '("Procter & Gamble" OR "Procter and Gamble" OR "P&G stock" OR "consumer goods company")',
    "JNJ": '("Johnson & Johnson" OR "Johnson and Johnson" OR "Johnson & Johnson stock" OR "healthcare company")',
    "PEP": '("PepsiCo" OR "PepsiCo Inc" OR "PepsiCo stock" OR "Pepsi beverages")',
    "WMT": '("Walmart" OR "Walmart Inc" OR "Walmart stock" OR "retail giant")',
    "MCD": '("McDonald\'s" OR "McDonalds Corporation" OR "McDonald\'s stock" OR "fast food chain")',
    "VZ": '("Verizon" OR "Verizon Communications" OR "Verizon stock" OR "telecommunications company")',
    "DUK": '("Duke Energy" OR "Duke Energy Corporation" OR "utility company stock")',
    "UL": '("Unilever" OR "Unilever PLC" OR "consumer goods multinational")',
    "V": '("Visa" OR "Visa Inc" OR "Visa stock" OR "payment network company")',
    "SPY": '("SPDR S&P 500 ETF" OR "SPY ETF" OR "S&P 500 ETF")',
    "QQQ": '("Invesco QQQ ETF" OR "Nasdaq 100 ETF" OR "QQQ ETF")',
    "EEM": '("Emerging Markets ETF" OR "EEM ETF")',
    "VGK": '("Vanguard FTSE Europe ETF" OR "VGK ETF")',
    "AGG": '("US bond market ETF" OR "AGG ETF")',
    "VNQ": '("US real estate ETF" OR "VNQ ETF")',
    "ARKK": '("ARK Innovation ETF" OR "ARKK ETF")',
    "VUG": '("Vanguard Growth ETF" OR "VUG ETF")',
    "SCHD": '("US dividend ETF" OR "SCHD ETF")',
    "SOXX": '("Semiconductor ETF" OR "SOXX ETF")',
    "EURUSD": '("EUR USD forex" OR "Euro dollar exchange rate")',
    "USDJPY": '("USD JPY forex" OR "Dollar Yen exchange rate")',
    "GBPUSD": '("GBP USD forex" OR "Pound Dollar exchange rate")',
    "USDCHF": 'USD CHF forex',
    "AUDUSD": 'AUD USD forex',
    "USDCAD": 'USD CAD forex',
    "NZDUSD": 'NZD USD forex',
    "EURGBP": 'EUR GBP forex',
    "EURJPY": 'EUR JPY forex',
    "GBPJPY": 'GBP JPY forex',
    "BTC-USD": '("Bitcoin" OR "Bitcoin price" OR "Bitcoin cryptocurrency")',
    "ETH-USD": '("Ethereum" OR "Ethereum price" OR "Ethereum cryptocurrency")',
    "BNB-USD": '("Binance Coin" OR "BNB cryptocurrency")',
    "XRP-USD": '("Ripple" OR "XRP cryptocurrency")',
    "SOL-USD": '("Solana" OR "Solana cryptocurrency")',
    "TRX-USD": '("TRON")',
    "DOGE-USD": '("Dogecoin" OR "Dogecoin cryptocurrency")',
    "ADA-USD": '("Cardano" OR "ADA cryptocurrency")',
    "AVAX-USD": '("Avalanche" OR "AVAX cryptocurrency")',
    "LTC-USD": '("Litecoin" OR "Litecoin cryptocurrency")',
    "GLD": '("Gold price" OR "Gold ETF")',
    "SLV": '("Silver price" OR "Silver ETF")',
    "PPLT": '("Platinum price" OR "Platinum ETF")',
    "PALL": '("Palladium price" OR "Palladium ETF")',
    "USO": '("Oil price" OR "Oil ETF")',
    "UNG": '("Natural gas price" OR "Natural gas ETF")',
    "CORN": '("Corn price" OR "Corn ETF" OR "agricultural commodities")'
}


def build_query(user_input: str) -> str:
    s = user_input.strip()
    key = s.upper().replace(" ", "")

    if key in ALIASES:
        return ALIASES[key]

    if s.isupper() and len(s) <= 6:
        return f'("{s}" AND (stock OR shares OR earnings OR market))'

    if " " in s:
        return f'("{s}" AND (stock OR shares OR earnings OR market))'

    return s


### Construcción de URLs para feeds RSS (query final)

La función `make_rss_query_from_base` recibe una URL base de RSS (por ejemplo, de Google News) y una `query` ya procesada:

- Ensambla la URL completa con parámetros codificados correctamente.
- Garantiza que los operadores lógicos y caracteres especiales se transmiten en un formato aceptado por el proveedor RSS.

El resultado es una URL lista para ser consumida por `feedparser`, integrando así la capa de construcción de consultas con la capa de obtención de datos.

In [0]:
def make_rss_query_from_base(base: str) -> str:
    """
    Convierte una base tipo GDELT (ORs) en una query simple para Google News RSS.
    """
    q = base
    q = re.sub(r"\bsourcelang:\w+\b", "", q, flags=re.IGNORECASE)
    q = q.replace("(", " ").replace(")", " ")
    q = re.sub(r"\s+", " ", q).strip()
    return q

### Carga del modelo de sentimiento financiero (FinBERT)

Aquí se inicializa un `pipeline` de `transformers` para la tarea de `sentiment-analysis` utilizando el modelo:

- `ProsusAI/finbert`, una variante de BERT especializada en **texto financiero**.

Este componente es clave en el TFM, ya que convierte textos de noticias en etiquetas de sentimiento (`positive`, `neutral`, `negative`) con una probabilidad asociada, lo que más adelante se traducirá en una señal numérica.

In [0]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert",
    tokenizer="ProsusAI/finbert"
)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/torch/_vmap_internals.py:9: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  from torch.utils._pytree import _broadcast_to_and_flatten, tree_flatten, tree_unflatten


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: ProsusAI/finbert
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### Comprobación de versiones de `torch` y `transformers`

Esta celda imprime las versiones instaladas de:

- `torch`
- `transformers`

Para este caso se tiene que tener una versión de torch arriba de 2.6

In [0]:
import torch, transformers
print(torch.__version__)
print(transformers.__version__)

2.10.0+cu128
5.1.0


### Construcción del texto de entrada para el modelo de sentimiento

La función `build_text_for_sentiment` define cómo se combina la información de cada noticia para alimentar el modelo de análisis de sentimiento:

- Prioriza el uso conjunto de `title` + `summary` cuando ambos están disponibles.
- Si solo hay título, utiliza únicamente ese campo.

Esta decisión controla el **contexto textual** con el que FinBERT evalúa cada noticia y, por tanto, influye en la clasificación de sentimiento que se obtiene.


In [0]:
def build_text_for_sentiment(item: dict) -> str:
    title = (item.get("title") or "").strip()
    summary = (item.get("summary") or "").strip()
    if summary:
        return f"{title}. {summary}"
    return title

In [0]:
ASSETS = list(ALIASES.keys())   # o pon una sublista si quieres
START_DATE = "2026-02-14"
END_DATE   = "2026-02-15"

OUTPUT_DIR = "sentiment_daily"


In [0]:
from datetime import datetime, timedelta

def iter_days(start_date: str, end_date: str):
    start = datetime.strptime(start_date, "%Y-%m-%d").date()
    end = datetime.strptime(end_date, "%Y-%m-%d").date()

    current = start
    while current <= end:
        yield current.strftime("%Y-%m-%d")
        current += timedelta(days=1)


In [0]:
OUTPUT_PATH = "abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/config/news"

In [0]:
# dbutils.fs.rm("abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/config/news", recurse=True)



In [0]:
import os

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [0]:
import time
import pandas as pd

LABEL_TO_NUM = {"positive": 1, "neutral": 0, "negative": -1}

for asset in ASSETS:
    for day in iter_days(START_DATE, END_DATE):

        print(f"\nProcesando {asset} | {day}")

        try:
            out = fetch_news_gdelt_by_date_range(
                user_input=asset,
                start_date=day,
                end_date=day,
                n=50000,   # overfetch
            )

        except Exception as e:
            print(f"  ❌ Error en GDELT: {e}")
            time.sleep(5)
            continue

        items = out.get("items", [])
        if not items:
            print("  ⚠️ Sin noticias")
            time.sleep(3)
            continue

        # ------------------------
        # Sentimiento por noticia
        # ------------------------
        for item in items:
            text = build_text_for_sentiment(item)
            if not text:
                item["sentiment"] = None
                item["sentiment_score"] = None
                item["sentiment_value"] = None
                continue

            pred = sentiment_model(text)[0]
            item["sentiment"] = pred["label"]
            item["sentiment_score"] = float(pred["score"])
            item["sentiment_value"] = LABEL_TO_NUM[pred["label"]] * float(pred["score"])

        # ------------------------
        # ------------------------
        # Guardar parquet blindado
        # ------------------------

        df = pd.DataFrame(items)

        if df.empty:
            print(f"  ⚠️ Sin datos para {asset} {day}")
        else:

            # Asegurar columnas
            expected_cols = [
                "published_utc",
                "published_local",
                "title",
                "link",
                "provider",
                "sentiment",
                "sentiment_score",
                "sentiment_value"
            ]

            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # Añadir particiones
            df["asset"] = asset
            df["date"] = day

            # 🔥 Limpieza fuerte de tipos
            df["published_utc"] = pd.to_datetime(df["published_utc"], errors="coerce")
            df["published_local"] = pd.to_datetime(df["published_local"], errors="coerce")

            df["sentiment_score"] = pd.to_numeric(df["sentiment_score"], errors="coerce")
            df["sentiment_value"] = pd.to_numeric(df["sentiment_value"], errors="coerce")

            string_cols = ["title","link","provider","sentiment","asset","date"]
            for c in string_cols:
                df[c] = df[c].astype(str)

            # 🔥 ORDEN EXPLÍCITO de columnas
            df = df[
                [
                    "published_utc",
                    "published_local",
                    "title",
                    "link",
                    "provider",
                    "sentiment",
                    "sentiment_score",
                    "sentiment_value",
                    "asset",
                    "date"
                ]
            ]

            from pyspark.sql.types import *

            schema = StructType([
                StructField("published_utc", TimestampType(), True),
                StructField("published_local", TimestampType(), True),
                StructField("title", StringType(), True),
                StructField("link", StringType(), True),
                StructField("provider", StringType(), True),
                StructField("sentiment", StringType(), True),
                StructField("sentiment_score", DoubleType(), True),
                StructField("sentiment_value", DoubleType(), True),
                StructField("asset", StringType(), True),
                StructField("date", StringType(), True),
            ])

            from delta.tables import DeltaTable

            spark_df = spark.createDataFrame(df, schema=schema)

            if not DeltaTable.isDeltaTable(spark, OUTPUT_PATH):
                (
                    spark_df.write
                        .format("delta")
                        .mode("overwrite")
                        .partitionBy("asset","date")
                        .save(OUTPUT_PATH)
                )
            else:
                delta_table = DeltaTable.forPath(spark, OUTPUT_PATH)

                (
                    delta_table.alias("target")
                    .merge(
                        spark_df.alias("source"),
                        "target.link = source.link AND target.asset = source.asset"
                    )
                    .whenMatchedUpdateAll()
                    .whenNotMatchedInsertAll()
                    .execute()
                )

            print(f"  ✅ Delta MERGE completado: asset={asset} date={day}")




        # ⏸️ descanso para no romper GDELT
        time.sleep(4)



Procesando TSLA | 2026-02-14
  ✅ Delta MERGE completado: asset=TSLA date=2026-02-14

Procesando TSLA | 2026-02-15
  ✅ Delta MERGE completado: asset=TSLA date=2026-02-15

Procesando NVDA | 2026-02-14
  ✅ Delta MERGE completado: asset=NVDA date=2026-02-14

Procesando NVDA | 2026-02-15
  ✅ Delta MERGE completado: asset=NVDA date=2026-02-15

Procesando AMD | 2026-02-14
  ❌ Error en GDELT: HTTPSConnectionPool(host='api.gdeltproject.org', port=443): Read timed out. (read timeout=20)

Procesando AMD | 2026-02-15
  ✅ Delta MERGE completado: asset=AMD date=2026-02-15

Procesando COIN | 2026-02-14
  ✅ Delta MERGE completado: asset=COIN date=2026-02-14

Procesando COIN | 2026-02-15
  ✅ Delta MERGE completado: asset=COIN date=2026-02-15

Procesando PLTR | 2026-02-14
  ✅ Delta MERGE completado: asset=PLTR date=2026-02-14

Procesando PLTR | 2026-02-15
  ✅ Delta MERGE completado: asset=PLTR date=2026-02-15

Procesando RIVN | 2026-02-14
  ✅ Delta MERGE completado: asset=RIVN date=2026-02-14

Procesan

In [0]:
        #     spark_df = spark.createDataFrame(df, schema=schema)

        #     # ------------------------
        #     # Eliminar partición antes de reescribir
        #     # ------------------------

        #     partition_path = f"{OUTPUT_PATH}/asset={asset}/date={day}"

        #     try:
        #         dbutils.fs.rm(partition_path, recurse=True)
        #         print(f"  🗑️ Eliminada partición previa: asset={asset} date={day}")
        #     except Exception as e:
        #         print(f"  ℹ️ No existía partición previa para {asset} {day}")


        #     (
        #         spark_df.write
        #             .mode("append")
        #             .format("parquet")
        #             .partitionBy("asset", "date")
        #             .save(OUTPUT_PATH)
        #     )

        #     print(f"  ✅ Guardado parquet: asset={asset} date={day} ({spark_df.count()} filas)")



        # # ⏸️ descanso para no romper GDELT
        # time.sleep(4)

In [0]:
# output_path = "abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/config/news_202602"

# df = (
#     spark.read
#     .option("header", "true")      # ajusta si tus CSV no tienen header
#     .option("inferSchema", "true") # opcional, recomendado
#     .csv(f"{OUTPUT_DIR}/*.csv")
# )

# df.write.mode("overwrite").parquet(output_path)